In [ ]:
%load_ext autoreload
%autoreload 2

# Burger's 1D equation

Benchmark on the Burger 1D as proposed in the PINNacle benchmark paper (https://arxiv.org/pdf/2306.08827) (see Appendix B1: (**Burgers1d**)).

We dig through the PINNacle code database to retrieve the experimental setting (number of collocation points, number of iterations, ...) as well as the reference values of the solution.
While PINNacle codes use a callback for evaluating the error with respect to the reference, we simply make a pridction at the end of the training (see [here](https://github.com/i207M/PINNacle/blob/main/src/utils/callbacks.py#L104))

In [ ]:
seed = 1

In [ ]:
import numpy as np

# from PINNACLE
DEFAULT_NUM_DOMAIN_POINTS = 8192 // 2
DEFAULT_NUM_BOUNDARY_POINTS = 2048 // 2
DEFAULT_NUM_INITIAL_POINTS = 2048 // 2

ref_data = np.loadtxt("burgers1d.dat", comments="%").astype(np.float32)

**Summary:**
- [DeepXDE](#DeepXDE)
- [jinns](#jinns)
- [PINA](#PINA)
- [Comparisons](#Comparisons)

## DeepXDE


We use the JAX backend.

In [ ]:
import os
os.environ["DDE_BACKEND"]="pytorch"
import deepxde as dde

# Determinism may reduce performance in DeepXDE (see doc) but needs to
# be set for a fair comparison with Jinns
# https://deepxde.readthedocs.io/en/stable/modules/deepxde.html#deepxde.config.set_random_seed
dde.config.set_random_seed(seed)

# PDE for JAX
# def pde(x, y):
#     y_val, y_fn = y
#     dy_x, _ = dde.grad.jacobian(y, x, i=0, j=0)
#     dy_t, _ = dde.grad.jacobian(y, x, i=0, j=1)
#     dy_xx, _ = dde.grad.hessian(y, x, i=0, j=0)
#     return dy_t + y_val * dy_x - 0.01 / np.pi * dy_xx

# PDE for pytorch
def pde(x, y):
    dy_x = dde.grad.jacobian(y, x, i=0, j=0)
    dy_t = dde.grad.jacobian(y, x, i=0, j=1)
    dy_xx = dde.grad.hessian(y, x, i=0, j=0)
    return dy_t + y * dy_x - 0.01 / np.pi * dy_xx

geom = dde.geometry.Interval(-1, 1)
timedomain = dde.geometry.TimeDomain(0, 1)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)

bc = dde.icbc.DirichletBC(geomtime, lambda x: 0, lambda _, on_boundary: on_boundary)
ic = dde.icbc.IC(
    geomtime, lambda x: -np.sin(np.pi * x[:, 0:1]), lambda _, on_initial: on_initial
)

data = dde.data.TimePDE(
    geomtime,
    pde,
    [bc, ic],
    num_domain=DEFAULT_NUM_DOMAIN_POINTS,
    num_boundary=DEFAULT_NUM_BOUNDARY_POINTS,
    num_initial=DEFAULT_NUM_INITIAL_POINTS,
    # train_distribution="uniform" # this is the default
    # num_test=None # this is the Default, we perform a evaluation callback on the same dataset as the training (see the test column in the training log)
)
net = dde.nn.FNN([2] + [100] * 5 + [1], "tanh", "Glorot normal")
model = dde.Model(data, net)

model.compile("adam", lr=1e-4)
losshistory, train_state = model.train(iterations=40000)
dde.saveplot(losshistory, train_state, issave=True, isplot=True)

## jinns

In [ ]:
import jax
import equinox as eqx
import jinns
import jax.numpy as jnp
from jax import grad, vmap, random
import optax
import time
import matplotlib.pyplot as plt

key = random.PRNGKey(seed)

In [ ]:
eqx_list = (
    (eqx.nn.Linear, 2, 100),
    (jax.nn.tanh,),
    (eqx.nn.Linear, 100, 100),
    (jax.nn.tanh,),
    (eqx.nn.Linear, 100, 100),
    (jax.nn.tanh,),
    (eqx.nn.Linear, 100, 100),
    (jax.nn.tanh,),
    (eqx.nn.Linear, 100, 100),
    (jax.nn.tanh,),
    (eqx.nn.Linear, 100, 1)
)
key, subkey = random.split(key)
u_pinn, init_nn_params_pinn = jinns.utils.create_PINN(subkey, eqx_list, "nonstatio_PDE", 1)

In [ ]:
n = DEFAULT_NUM_DOMAIN_POINTS
ni = DEFAULT_NUM_INITIAL_POINTS
nb = DEFAULT_NUM_BOUNDARY_POINTS
dim = 1
xmin = -1
xmax = 1
tmin = 0
tmax = 1
method = "grid" # equals "uniform" in deepXDE, while "uniform" in jinns equals "pseudo" in deepXDE

train_data = jinns.data.CubicMeshPDENonStatio(
    key=subkey,
    n=n,
    nb=nb,
    ni=ni,
    dim=dim,
    min_pts=(xmin,),
    max_pts=(xmax,),
    tmin=tmin,
    tmax=tmax,
    method=method,
)

In [ ]:
# create Burger loss
nu = 1 / (100 * jnp.pi)


In [ ]:
# Init parameters for NN and equation
init_params_pinn = jinns.parameters.Params(
    nn_params=init_nn_params_pinn,
    eq_params={
        "nu":nu
    }
)

In [ ]:
from jinns.loss import PDENonStatio

class BurgerEquation(PDENonStatio):
    def equation(self, t_x, u, params):
        u_ = lambda t_x: jnp.squeeze(u(t_x, params))
        du_dtx = jax.grad(u_)
        d2u_dx_dtx = jax.grad(lambda t_x: du_dtx(t_x)[1])
        du_dtx_values = du_dtx(t_x)

        return du_dtx_values[0:1] + (
            u_(t_x) * du_dtx_values[1:2]
            - params.eq_params["nu"] * d2u_dx_dtx(t_x)[1:2]
        )

be_loss = BurgerEquation()

In [ ]:
def u0(x):
    return -jnp.sin(jnp.pi * x)

loss_weights = jinns.loss.LossWeightsPDENonStatio(
    dyn_loss=10, initial_condition=10, boundary_loss=1
)

loss_pinn = jinns.loss.LossPDENonStatio(
    u=u_pinn,
    loss_weights=loss_weights,
    dynamic_loss=be_loss,
    omega_boundary_fun=lambda t_dx: 0,
    omega_boundary_condition="dirichlet",
    initial_condition_fun=u0,
    params=init_params_pinn,
)

In [ ]:
# Training loop in Jinns
params_pinn = init_params_pinn
tx = optax.adam(learning_rate=1e-4)
n_iter = 40000

In [ ]:
params_pinn, total_loss_list_pinn, loss_by_term_dict_pinn, _, _, _, _ , _, _ = jinns.solve(
    init_params=params_pinn,
    data=train_data,
    optimizer=tx,
    loss=loss_pinn,
    n_iter=n_iter,
    verbose=True
)

In [ ]:
for loss_name, loss_values in loss_by_term_dict_pinn.items():
    plt.plot(jnp.log10(loss_values), label=loss_name)
plt.plot(jnp.log10(total_loss_list_pinn), label="total loss")
plt.legend()
plt.show()

In [ ]:
nx = 500
val_xdata = jnp.linspace(xmin, xmax, nx)
ntime = 500
val_times = jnp.linspace(tmin, tmax, ntime)

u_est_pinn = lambda t_x:u_pinn(t_x, params_pinn)
# same subkey
jinns.plot.plot1d_image(u_est_pinn, xdata=val_xdata, times=val_times, cmap="viridis", colorbar=True, figsize=(5, 5), title="u(t, x)")

In [ ]:
time_slices = [0, 0.2, 0.4, 0.75, 0.95, 1.]
jinns.plot.plot1d_slice(u_est_pinn, xdata=val_xdata, time_slices=time_slices,  figsize=(5, 5))

In [ ]:
from functools import partial

cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
diffs = []
for i, ti in enumerate(time_slices):
    val_ti = jnp.column_stack([val_xdata, jnp.repeat(ti, nx)])
    val_ti_normalized = jnp.column_stack([jnp.repeat(ti, nx), val_xdata, ])
    u_est_tfixed = vmap(u_est_pinn)#vmap(partial(u_est_pinn, t=ti * jnp.ones((1,)) / Tmax), 0, 0)
    plt.plot(val_xdata, model.predict(x=val_ti), label=rf"$DDE: t_i = {ti}$", linestyle="--", alpha=.7, color = cycle[i])
    plt.plot(val_xdata, u_est_tfixed(val_ti_normalized), label=rf"$Jinns:  t_i = {ti}$", alpha=.7, color = cycle[i])
    diffs.append(jnp.abs(u_est_tfixed(val_ti_normalized) - model.predict(x=val_ti)))

    # plt.plot(val_xdata, u_est_tfixed(x=val_xdata[:, None]), label=rf"$Jinns:  t_i = {ti}$", alpha=.7, color = cycle[i])
    # diffs.append(jnp.abs(u_est_tfixed(x=val_xdata[:, None]) - model.predict(x=val_ti)))

from matplotlib.lines import Line2D
line_jinns = Line2D([0], [0], label='Jinns', color='black',linestyle="-")
line_dde = Line2D([0], [0], label='DeepXDE', color='black', linestyle="--")
plt.legend(handles=[line_jinns, line_dde])

In [ ]:
for dif, ti in zip(diffs, time_slices):
    plt.plot(val_xdata, dif, label=rf"x-differences at time $t_i={ti}$")
plt.legend()

## PINA

In [ ]:
from pytorch_lightning import seed_everything

# setting the seed for reproducibility
seed_everything(seed, workers=True)

In [ ]:
from pina.problem import SpatialProblem, TimeDependentProblem
from pina.equation import Equation, FixedValue, FixedGradient, FixedFlux
from pina.geometry import CartesianDomain
import torch
from pina.operators import grad, laplacian
from pina import Condition

In [ ]:
class Burgers1DEquation(Equation):

    def __init__(self, nu = 0.):
        """
        Burgers1D class. This class can be
        used to enforce the solution u to solve the viscous Burgers 1D Equation.

        :param torch.float32 nu: the viscosity coefficient. Default value is set to 0.
        """
        self.nu = nu

        def equation(input_, output_):
                return grad(output_, input_, d='t') +\
                       output_*grad(output_, input_, d='x') -\
                       self.nu*laplacian(output_, input_, d='x')


        super().__init__(equation)

In [ ]:
class Burgers1D(TimeDependentProblem, SpatialProblem):

    # define initial condition
    def initial_condition(input_, output_):
        u_expected = -torch.sin(torch.pi*input_.extract(['x']))
        return output_.extract(['u']) - u_expected

    # assign output/ spatial and temporal variables
    output_variables = ['u']
    spatial_domain = CartesianDomain({'x': [-1, 1]})
    temporal_domain = CartesianDomain({'t': [0, 1]})

    # problem condition statement
    conditions = {
        'gamma1': Condition(location=CartesianDomain({'x': -1, 't': [0, 1]}), equation=FixedValue(0.)),
        'gamma2': Condition(location=CartesianDomain({'x':  1, 't': [0, 1]}), equation=FixedValue(0.)),
        't0': Condition(location=CartesianDomain({'x': [-1, 1], 't': 0}), equation=Equation(initial_condition)),
        'D': Condition(location=CartesianDomain({'x': [-1, 1], 't': [0, 1]}), equation=Burgers1DEquation(0.01/torch.pi)),
    }
burgers1D = Burgers1D()

In [ ]:
from math import sqrt

burgers1D.discretise_domain(
    n=int(sqrt(DEFAULT_NUM_DOMAIN_POINTS)), # NOTE the sqrt
    mode='grid',
    variables=['x', 't'],
    locations='D'
)
burgers1D.discretise_domain(
    n=DEFAULT_NUM_BOUNDARY_POINTS // 2, # divide by 2 since NUM_BOUNDARY is the total
    mode='grid',
    variables=['x', 't'],
    locations=['gamma1', 'gamma2']
)
burgers1D.discretise_domain(
    n=DEFAULT_NUM_INITIAL_POINTS,
    mode='grid',
    variables=['x', 't'],
    locations=['t0']
)
print('Input points:', burgers1D.input_pts['D'].shape)
print('Input points:', burgers1D.input_pts['gamma2'].shape)
print('Input points:', burgers1D.input_pts['gamma1'].shape)
print('Input points:', burgers1D.input_pts['t0'].shape)

In [ ]:
from pina import Plotter

# pl = Plotter()
# pl.plot_samples(problem=burgers1D)

In [ ]:
from pina import Trainer
from pina.solvers import PINN
from pina.model import FeedForward
from pina.callbacks import MetricTracker, PINAProgressBar
from pytorch_lightning.callbacks import Timer

pbar = PINAProgressBar(metrics=['mean'], refresh_rate=1)

# build the model
model_ = FeedForward(
    layers=[100, 100, 100, 100, 100],
    func=torch.nn.Tanh,
    output_dimensions=len(burgers1D.output_variables),
    input_dimensions=len(burgers1D.input_variables)
)

# create the PINN object
pinn = PINN(
    burgers1D,
    model_,
    optimizer=torch.optim.Adam,
    optimizer_kwargs={'lr': 1e-4}
)

# create the trainer
trainer = Trainer(
    solver=pinn,
    max_epochs=40000,
    callbacks=[MetricTracker(), pbar, Timer()],
    accelerator='gpu',
    enable_model_summary=False,
    log_every_n_steps=1000,
    val_check_interval=1000,
    check_val_every_n_epoch=None,
    enable_checkpointing=False
) 

# train
trainer.train()

print(f'Total training time {trainer.callbacks[-1].time_elapsed("train"):.5f} s')

In [ ]:
pl = Plotter()
pl.plot(solver=pinn)

In [ ]:
pl.plot_loss(trainer=trainer, label = 'mean_loss', logy=True)

## Comparisons

In [ ]:
import sys
sys.path.append('../')

from PINNacle_utils import trans_time_data_to_dataset, compute_relative_errors

ref_data_ = trans_time_data_to_dataset(ref_data, "burgers1d.dat", 2, 1)

nan_mask = np.isnan(ref_data_).any(axis=1)
test_x = ref_data_[~nan_mask, :2]
test_y = ref_data_[~nan_mask, 2:]

test_x_inv = np.copy(test_x)
test_x_inv[:, 0] = test_x[:, 1]
test_x_inv[:, 1] = test_x[:, 0]

In [ ]:
compute_relative_errors(test_y, model.predict(test_x))

In [ ]:
# /!\ t and x must be inverted in jinns convention

compute_relative_errors(test_y, jax.vmap(u_est_pinn)(test_x_inv))

In [ ]:
from pina import LabelTensor
compute_relative_errors(
    test_y,
    pinn(LabelTensor(test_x_inv, ['t', 'x'])).detach().cpu().numpy()
)